# Routes

> SSE route helpers and decorators for FastHTML

In [ ]:
#| default_exp core.routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import asyncio
from typing import Optional, Callable, Any, Dict, AsyncGenerator, Union
from functools import wraps
from dataclasses import dataclass
from fasthtml.common import EventStream, sse_message
from cjm_fasthtml_sse.core.connections import ConnectionRegistry
from cjm_fasthtml_sse.core.streaming import SSEStream, StreamConfig

## Core Classes

In [ ]:
#| export
@dataclass
class SSERouteConfig:
    """Configuration for SSE route"""
    connection_type: str = "sse"  # Connection type for registry
    stream_config: Optional[StreamConfig] = None  # Stream configuration
    registry: Optional[ConnectionRegistry] = None  # Connection registry
    validate_fn: Optional[Callable] = None  # Validation function
    metadata_fn: Optional[Callable] = None  # Function to generate metadata
    error_handler: Optional[Callable] = None  # Error handler function
    debug: bool = False  # Enable debug logging

In [ ]:
#| export
def create_sse_route(
    data_generator: Callable,  # Async generator function
    transform_fn: Optional[Callable] = None,  # Optional transform function
    config: Optional[SSERouteConfig] = None  # Route configuration
) -> Callable:  # Route handler
    """Create an SSE route handler with automatic connection management."""
    
    config = config or SSERouteConfig()
    
    async def route_handler(**kwargs):
        """SSE route handler."""
        
        # Validation
        if config.validate_fn:
            validation_result = await config.validate_fn(**kwargs) if asyncio.iscoroutinefunction(config.validate_fn) else config.validate_fn(**kwargs)
            if not validation_result:
                return EventStream(async_error_generator("Validation failed"))
        
        async def stream():
            connection = None
            
            try:
                # Register connection if registry provided
                if config.registry:
                    metadata = config.metadata_fn(**kwargs) if config.metadata_fn else kwargs
                    connection = await config.registry.add_connection(
                        conn_type=config.connection_type,
                        metadata=metadata
                    )
                    if config.debug:
                        print(f"[SSERoute] Connection registered: {connection.connection_id}")
                
                # Generate data
                data_source = data_generator(**kwargs)
                if callable(data_source) and not hasattr(data_source, '__anext__'):
                    data_source = data_source()
                
                # Stream with SSEStream wrapper
                stream_config = config.stream_config or StreamConfig()
                sse_stream = SSEStream(stream_config)
                
                async for message in sse_stream.stream(data_source, transform_fn):
                    yield message
                    
            except Exception as e:
                if config.error_handler:
                    error_msg = await config.error_handler(e, **kwargs) if asyncio.iscoroutinefunction(config.error_handler) else config.error_handler(e, **kwargs)
                    yield sse_message(error_msg)
                elif config.debug:
                    yield f": error {str(e)}\n\n"
                raise
                
            finally:
                # Unregister connection
                if connection and config.registry:
                    await config.registry.remove_connection(connection.connection_id)
                    if config.debug:
                        print(f"[SSERoute] Connection unregistered: {connection.connection_id}")
        
        return EventStream(stream())
    
    return route_handler

async def async_error_generator(message: str):
    """Generate an error message for SSE."""
    yield f": error {message}\n\n"

In [ ]:
#| export
def sse_route(
    path: Optional[str] = None,  # Route path
    transform_fn: Optional[Callable] = None,  # Transform function
    config: Optional[SSERouteConfig] = None  # Route configuration
) -> Callable:  # Decorator
    """Decorator to create SSE routes."""
    
    def decorator(func: Callable) -> Callable:
        """Decorate an async generator function as an SSE route."""
        
        # Create the SSE route handler
        handler = create_sse_route(func, transform_fn, config)
        
        # Preserve function metadata
        handler.__name__ = func.__name__
        handler.__doc__ = func.__doc__
        
        # Store path for app registration
        handler.sse_path = path or f"/{func.__name__}"
        
        return handler
    
    return decorator

### Tests for SSE Route

In [ ]:
#| test
# Test SSERouteConfig
config = SSERouteConfig(
    connection_type="test",
    debug=True
)

assert config.connection_type == "test"
assert config.debug == True
assert config.stream_config is None
assert config.registry is None
assert config.validate_fn is None

print("✓ SSERouteConfig tests passed")

✓ SSERouteConfig tests passed


In [ ]:
#| test
# Test create_sse_route
async def test_data_generator(param1="default"):
    for i in range(3):
        yield f"message_{i}_{param1}"
        await asyncio.sleep(0.01)

# Create route without config
handler = create_sse_route(test_data_generator)
assert callable(handler)

# Create route with transform
def transform(data):
    return f"Transformed: {data}"

handler_with_transform = create_sse_route(
    test_data_generator,
    transform_fn=transform
)
assert callable(handler_with_transform)

print("✓ create_sse_route tests passed")

✓ create_sse_route tests passed


In [ ]:
#| test
# Test sse_route decorator
@sse_route(path="/test_stream")
async def test_stream(value="test"):
    for i in range(2):
        yield f"{value}_{i}"

assert callable(test_stream)
assert test_stream.sse_path == "/test_stream"
assert test_stream.__name__ == "test_stream"

# Test with transform
@sse_route(transform_fn=lambda x: {"data": x})
async def test_stream_transform():
    yield "test"

assert callable(test_stream_transform)
assert test_stream_transform.sse_path == "/test_stream_transform"

print("✓ sse_route decorator tests passed")

✓ sse_route decorator tests passed


## Helper Functions

In [ ]:
#| export
def setup_sse_routes(
    app,  # FastHTML app
    *handlers,  # SSE route handlers created with @sse_route
    prefix: str = "",  # URL prefix
    registry: Optional[ConnectionRegistry] = None  # Shared registry
):
    """Setup multiple SSE routes on an app."""
    for handler in handlers:
        if hasattr(handler, 'sse_path'):
            path = f"{prefix}{handler.sse_path}"
            
            # If handler needs registry and doesn't have one, inject it
            if registry and not hasattr(handler, '_registry'):
                handler._registry = registry
            
            app.route(path)(handler)
            print(f"[SSE Routes] Registered: {path}")

In [ ]:
#| export
def create_conditional_sse_route(
    active_generator: Callable,  # Generator for active entities
    inactive_content: Callable,  # Function to generate content for inactive entities
    is_active_fn: Callable,  # Function to check if entity is active
    transform_fn: Optional[Callable] = None,  # Transform function
    config: Optional[SSERouteConfig] = None  # Route configuration
) -> Callable:  # Route handler
    """Create an SSE route that handles both active and inactive states."""
    
    async def conditional_generator(**kwargs):
        """Conditional data generator."""
        # Check if entity is active
        is_active = await is_active_fn(**kwargs) if asyncio.iscoroutinefunction(is_active_fn) else is_active_fn(**kwargs)
        
        if is_active:
            # Use active generator
            async for data in active_generator(**kwargs):
                yield data
        else:
            # Send inactive content and close
            content = await inactive_content(**kwargs) if asyncio.iscoroutinefunction(inactive_content) else inactive_content(**kwargs)
            yield content
    
    return create_sse_route(conditional_generator, transform_fn, config)

In [ ]:
#| test
# Test create_conditional_sse_route
async def active_gen(id=""):
    for i in range(3):
        yield f"active_{i}"
        await asyncio.sleep(0.01)

def inactive_gen(id=""):
    return f"inactive_{id}"

def check_active(id=""):
    return id != "inactive"

conditional_handler = create_conditional_sse_route(
    active_generator=active_gen,
    inactive_content=inactive_gen,
    is_active_fn=check_active
)

assert callable(conditional_handler)

print("✓ create_conditional_sse_route tests passed")

✓ create_conditional_sse_route tests passed


## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()